# Notebook de demostración: validación de tablas finales

Este notebook muestra la utilización del módulo `bigdata_validator`, encargado de la validación de las tablas finales del proyecto de automatización de indicadores de Big Data (México).

La documentación no se encuentra aquí, sino en el repositorio de GitHub: https://github.com/r-leo/bigdata_validator


## 1. Instalación e importación

Instalar el módulo `bigdata_validator` e importar su clase `Validator`:

In [1]:
%%capture
!pip install git+https://github.com/r-leo/bigdata_validator.git

In [2]:
import bigdata_validator
from bigdata_validator import Validator

Adicionalmente se puede verificar que la versión instalada sea la más reciente (`1.0.0`):

In [3]:
print(bigdata_validator.__version__)

1.0.0


## 2. Creación de validadores

Instanciar los validadores necesarios para los indicadores de **alta frecuencia**:

In [ ]:
# Consumo
con = Validator('131_data_consumption_20240301_20250312.csv', region_isocode='COUNTRY_ISOCODE',
                region_name='COUNTRY_SHORT_SPANISH_NAME', name='SHORT_SPANISH_NAME')

# Inversión
inv = Validator('132_data_investment_20240201_20250220.csv', region_isocode='COUNTRY_ISOCODE',
                region_name='COUNTRY_SHORT_SPANISH_NAME', name='SHORT_SPANISH_NAME')

# Sector exterior
exs = Validator('133_data_external_20240101_20250124.csv', region_isocode='COUNTRY_ISOCODE',
                region_name='COUNTRY_SHORT_SPANISH_NAME', name='SHORT_SPANISH_NAME')

# Actividad sectorial
sea = Validator('134_data_sectoral_20240301_20250314.csv', region_isocode='COUNTRY_ISOCODE',
                region_name='COUNTRY_SHORT_SPANISH_NAME', name='SUBSECTOR_SHORT_SPA')

Igualmente para el caso de los indicadores de **alta granularidad**:

In [ ]:
# Consumo nacional
con_agg = Validator('231_data_consumption_aggregate_20240301_20250314.csv', region_isocode='COUNTRY_ISOCODE',
                    region_name='COUNTRY_SHORT_SPANISH_NAME', name='SHORT_SPANISH_NAME')

# Consumo por regiones
con_reg = Validator('232_data_consumption_regions_20250314_20250314.csv', region_isocode='ISOCODE_PAIS',
                    region_name='COUNTRY_SHORT_SPANISH_NAME', name='SHORT_SPANISH_NAME')

# Consumo por estados
con_sta = Validator('233_data_consumption_states_20240301_20250314.csv', region_isocode='ISOCODE_PAIS',
                    region_name='COUNTRY_SHORT_SPANISH_NAME', name='SHORT_SPANISH_NAME')

# Consumo por ciudades
con_cit = Validator('234_data_consumption_cities_20240301_20250314.csv', region_isocode='ISOCODE',
                    region_name='ISOCODE_SHORT_SPANISH_NAME', name='SHORT_SPANISH_NAME')

## 3. Proceso de validación automática

Para efectuar la validación de cada uno de los indicadores basta con llamar al método `.validate()` del validador correspondiente. Si únicamente se quisiera obtener el valor booleano de la validación dentro de un flujo de código, bastaría con llamar al método `.is_valid()` (véase la [documentación](https://github.com/r-leo/bigdata_validator?tab=readme-ov-file#23-validar-dentro-de-un-script-is_valid)).

**Resultados de las validaciones de los indicadores de alta frecuencia**

In [ ]:
# Consumo
con.validate()

No se encontraron errores.


In [ ]:
# Inversión
inv.validate()

No se encontraron errores.


In [ ]:
# Sector exterior
exs.validate()

No se encontraron errores.


In [ ]:
# Actividad sectorial
sea.validate()

No se encontraron errores.


**Resultados de las validaciones de los indicadores de alta granularidad**

In [ ]:
# Consumo nacional
con_agg.validate()

No se encontraron errores.


In [ ]:
# Consumo por regiones
con_reg.validate()

No se encontraron errores.


In [ ]:
# Consumo por estados
con_sta.validate()

Errores encontrados (26):

1	El registro ['N' 'MX-CAM' 'Aerolíneas'] no tiene datos para la última fecha.
2	El registro ['N' 'MX-COL' 'Aerolíneas'] no tiene datos para la última fecha.
3	El registro ['N' 'MX-GUA' 'Aerolíneas'] no tiene datos para la última fecha.
4	El registro ['N' 'MX-HID' 'Aerolíneas'] no tiene datos para la última fecha.
5	El registro ['N' 'MX-MOR' 'Aerolíneas'] no tiene datos para la última fecha.
6	El registro ['N' 'MX-NAY' 'Aerolíneas'] no tiene datos para la última fecha.
7	El registro ['N' 'MX-OAX' 'Aerolíneas'] no tiene datos para la última fecha.
8	El registro ['N' 'MX-SIN' 'Aerolíneas'] no tiene datos para la última fecha.
9	El registro ['N' 'MX-SLP' 'Aerolíneas'] no tiene datos para la última fecha.
10	El registro ['N' 'MX-TAB' 'Aerolíneas'] no tiene datos para la última fecha.
11	El registro ['N' 'MX-TLA' 'Aerolíneas'] no tiene datos para la última fecha.
12	El registro ['N' 'MX-VER' 'Aerolíneas'] no tiene datos para la última fecha.
13	El registro ['N' 'M

In [ ]:
# Consumo por ciudades
con_cit.validate()

No se encontraron errores.


## 4. Corrección de errores

El fichero que contiene los datos de **consumo por estados** no pasó satisfactoriamente el proceso de validación. El problema es que para una gran cantidad de fechas, la categoría de 'Aerolíneas' en Sinaloa no tiene datos. La solución propuesta es eliminar la categoría 'Aerolíneas' del fichero:

In [ ]:
import pandas as pd

consumption_states_original = pd.read_csv('233_data_consumption_states_20240301_20250314.csv', dtype=str, skip_blank_lines=False)
consumption_states_new = consumption_states_original.copy().query('SHORT_SPANISH_NAME != "Aerolíneas"')

consumption_states_new.to_csv('233_data_consumption_states_20240301_20250314_new.csv', index=False)

print('Filas perdidas:', len(consumption_states_original) - len(consumption_states_new))

Filas perdidas: 24258


La nueva tabla final corregida se pasa nuevamente por el proceso de validación para verificar que esta vez no devuelva ningún error:

In [ ]:
Validator('233_data_consumption_states_20240301_20250314_new.csv', region_isocode='ISOCODE_PAIS',
          region_name='COUNTRY_SHORT_SPANISH_NAME', name='SHORT_SPANISH_NAME').validate()

No se encontraron errores.


Con esto queda concluido el proceso de validación y corrección de los 8 ficheros muestra para México.